In [1]:
import pandas as pd
from scipy import stats
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import os
from tqdm import tqdm

In [5]:
rna_tissue_gtex = pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/rna_tissue_gtex.tsv')
#rna_tissue = pd.read_csv('/Users/xinyutang/Desktop/transcript_rna_tissue.tsv')

# GTEx_data

## Data preprocessing

### Mapping tissue_id to columns and mapping gene symbol to gene_id

In [ ]:
# Loading GTEx Gene TPM Data 
GTEx_Analysis = pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/GETxV8/GTEx_Analysis_TPM_V8.csv')

In [ ]:
# preprocess geneid
GTEx_Analysis = GTEx_Analysis.rename({'Unnamed: 0':'gene_id'},axis=1)
GTEx_Analysis['new_gene_id'] = GTEx_Analysis.gene_id.apply(lambda x: x.split('.')[0])
GTEx_Analysis2=GTEx_Analysis

In [ ]:
# Import GTExPortal data (Tissue Name and Tissue ID)
GTExPortal = pd.read_csv('/Users/xinyutang/Documents/HPA2//Data/GETxV8/GTExPortal.csv')
tissue_df = GTExPortal[['Tissue Sample ID','Tissue']]
tissue_dic = dict(zip(tissue_df['Tissue Sample ID'], tissue_df.Tissue))

In [ ]:
# preprocess column name
columns_name = GTEx_Analysis2.columns
columns_name = columns_name[1:-1]
assigned_name = []
i =1

for c in columns_name:
    c=c.replace('.','-')
    tissue_id = c.split('-SM')[0]
    
    if tissue_id in tissue_dic.keys():    
        
        tissue=tissue_dic[tissue_id]
        
        sample_id=c.split('-SM')[1]
    
        assigned_name.append(tissue+sample_id)
        
    else:
        assigned_name.append(c)
        i+=1

In [ ]:
assigned_name.insert(0, "gene_id")
assigned_name.append('new_gene_id')
GTEx_Analysis2.columns = assigned_name

In [6]:
GTEx_Analysis2=pd.read_csv('/Users/xinyutang/Documents/HPA2//Data/GETxV8/GTEx_Analysis_TPM_V8_edited.csv')

### Mapping gene symbol to gene id

#### TPM

In [7]:
#rna_tissue_gtex = pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/rna_tissue_gtex.tsv')
#rna_tissue_gtex_unique = rna_tissue_gtex[['Gene','Gene name']].drop_duplicates(keep='first')
GTEx_Analysis2

,Unnamed: 0,gene_id,Adipose - Subcutaneous-5GZZ7,Muscle - Skeletal-5EGHI,Artery - Tibial-5EGHJ,Artery - Coronary-5N9CS,Heart - Atrial Appendage-5GIEN,Adipose - Visceral (Omentum)-5EGHH,Uterus-5EGGH,Vagina-5GZY6,...,Adrenal Gland-5N9CK,Thyroid-5GZWS,Stomach-5GZZ6,Esophagus - Muscularis-5E43L,Ovary-5EGIU,Vagina-5EGIV,Artery - Tibial-5E44I,Muscle - Skeletal-5E45Y,Adipose - Subcutaneous-5NQ8O,new_gene_id
0,0,ENSG00000223972.5,0.00000,0.000,0.000,0.00000,0.000,0.0000,0.000,0.00000,...,0.03629,0.00000,0.000,0.00000,0.0000,0.000,0.00000,0.01965,0.02522,ENSG00000223972
1,1,ENSG00000227232.5,8.76400,3.861,7.349,11.07000,3.306,5.3890,11.990,16.95000,...,2.26800,5.38600,2.310,2.45600,4.0230,1.922,2.85700,0.86960,2.16700,ENSG00000227232
2,2,ENSG00000278267.1,0.00000,0.000,1.004,0.00000,0.000,0.0000,0.000,0.00000,...,0.00000,0.00000,0.000,0.00000,0.0000,0.000,0.00000,0.00000,0.00000,ENSG00000278267
3,3,ENSG00000243485.5,0.07187,0.000,0.000,0.06761,0.000,0.0000,0.000,0.00000,...,0.00000,0.06073,0.000,0.08464,0.1435,0.000,0.05216,0.00000,0.00000,ENSG00000243485
4,4,ENSG00000237613.2,0.00000,0.000,0.000,0.00000,0.000,0.0000,0.000,0.03904,...,0.00000,0.00000,0.000,0.00000,0.0000,0.000,0.00000,0.00000,0.00000,ENSG00000237613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56195,56195,ENSG00000198695.2,3799.00000,14750.000,6728.000,817.30000,7051.000,1857.0000,1041.000,1362.00000,...,7690.00000,14080.00000,17230.000,18710.00000,4290.0000,2446.000,16250.00000,19660.00000,9803.00000,ENSG00000198695
56196,56196,ENSG00000210194.1,16.24000,44.310,23.740,0.00000,2.252,3.7310,2.319,0.68970,...,31.82000,48.78000,65.860,65.86000,15.3100,4.530,58.26000,59.07000,34.76000,ENSG00000210194
56197,56197,ENSG00000198727.2,6938.00000,26310.000,8455.000,8799.00000,29170.000,18180.0000,9593.000,7064.00000,...,29300.00000,33570.00000,34040.000,31900.00000,20620.0000,16820.000,19200.00000,36670.00000,27740.00000,ENSG00000198727
56198,56198,ENSG00000210195.2,0.94300,6.414,1.034,0.00000,0.000,0.7801,0.000,0.00000,...,0.47530,0.79690,2.354,2.77600,0.0000,0.000,0.00000,47.86000,3.30400,ENSG00000210195


In [8]:
# Add missing genes
gtex_autophogy_geneid=pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/gtex_autophogy_geneid.csv')

In [14]:
# Filter out autophagy genes
GTEx_Analysis2 = GTEx_Analysis2.merge(gtex_autophogy_geneid, left_on='new_gene_id', right_on='Gene')
del GTEx_Analysis2['Gene']

In [21]:
selected_cols = GTEx_Analysis2.columns[~GTEx_Analysis2.columns.str.contains('SM',case=False)]
GTEx_Analysis3 = GTEx_Analysis2[selected_cols]

In [23]:
GTEx_Analysis3.to_csv('/Users/xinyutang/Documents/HPA2/Data/GETxV8/gtex_autophogy_TPM.csv')

In [24]:
#check result with Getx website --correct
ATG13_geneTPM=GTEx_Analysis3[GTEx_Analysis3['new_gene_id']=='ENSG00000175224']
filter_col = [col for col in GTEx_Analysis3 if col.startswith('Liver')]
np.median(ATG13_geneTPM[filter_col].values)

17.35

In [25]:
col = GTEx_Analysis3.columns.tolist()
col = col[1:-2]
autophagy_geneTPM = pd.melt(GTEx_Analysis3,id_vars =['gene_id','new_gene_id','GeneName'], value_vars =col,
                            var_name ='Tissue', value_name ='GETx_TPM')

In [26]:
autophagy_geneTPM

,gene_id,new_gene_id,GeneName,Tissue,GETx_TPM
0,ENSG00000125703.14,ENSG00000125703,ATG4C,Adipose - Subcutaneous-5GZZ7,5.3030
1,ENSG00000153898.12,ENSG00000153898,MCOLN2(TRPML2),Adipose - Subcutaneous-5GZZ7,3.5700
2,ENSG00000055732.12,ENSG00000055732,MCOLN1(TRPML3),Adipose - Subcutaneous-5GZZ7,9.9610
3,ENSG00000196289.7,ENSG00000196289,BECN2,Adipose - Subcutaneous-5GZZ7,0.2881
4,ENSG00000197769.5,ENSG00000197769,MAP1LC3C,Adipose - Subcutaneous-5GZZ7,26.4800
...,...,...,...,...,...
651787,ENSG00000078142.12,ENSG00000078142,PIK3C3,Adipose - Subcutaneous-5NQ8O,9.6290
651788,ENSG00000090674.15,ENSG00000090674,MCOLN1(TRPML1),Adipose - Subcutaneous-5NQ8O,47.6400
651789,ENSG00000130734.9,ENSG00000130734,ATG4D,Adipose - Subcutaneous-5NQ8O,12.0700
651790,ENSG00000101460.12,ENSG00000101460,MAP1LC3A,Adipose - Subcutaneous-5NQ8O,37.8400


In [27]:
def getTissue(x):
    a = x.split('-')[:-1]
    return ''.join(a)

In [28]:
autophagy_geneTPM.Tissue = autophagy_geneTPM.Tissue.apply(lambda x: getTissue(x))

In [29]:
autophagy_geneTPM

,gene_id,new_gene_id,GeneName,Tissue,GETx_TPM
0,ENSG00000125703.14,ENSG00000125703,ATG4C,Adipose Subcutaneous,5.3030
1,ENSG00000153898.12,ENSG00000153898,MCOLN2(TRPML2),Adipose Subcutaneous,3.5700
2,ENSG00000055732.12,ENSG00000055732,MCOLN1(TRPML3),Adipose Subcutaneous,9.9610
3,ENSG00000196289.7,ENSG00000196289,BECN2,Adipose Subcutaneous,0.2881
4,ENSG00000197769.5,ENSG00000197769,MAP1LC3C,Adipose Subcutaneous,26.4800
...,...,...,...,...,...
651787,ENSG00000078142.12,ENSG00000078142,PIK3C3,Adipose Subcutaneous,9.6290
651788,ENSG00000090674.15,ENSG00000090674,MCOLN1(TRPML1),Adipose Subcutaneous,47.6400
651789,ENSG00000130734.9,ENSG00000130734,ATG4D,Adipose Subcutaneous,12.0700
651790,ENSG00000101460.12,ENSG00000101460,MAP1LC3A,Adipose Subcutaneous,37.8400


In [30]:
autophagy_geneTPM = autophagy_geneTPM.rename({'GTEx_TPM':'TPM'},axis=1)
autophagy_geneTPM.to_csv('/Users/xinyutang/Documents/HPA2/Data/GETxV8/gtex_autophogy_TPM_final.csv')

#  Getting Url for Web Scraping

In [51]:
autophagy_geneTPM =pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/GETxV8/gtex_autophogy_TPM_final.csv')

In [52]:
autophagy_geneTPM

,Unnamed: 0,gene_id,new_gene_id,GeneName,Tissue,GETx_TPM
0,0,ENSG00000125703.14,ENSG00000125703,ATG4C,Adipose Subcutaneous,5.3030
1,1,ENSG00000153898.12,ENSG00000153898,MCOLN2(TRPML2),Adipose Subcutaneous,3.5700
2,2,ENSG00000055732.12,ENSG00000055732,MCOLN1(TRPML3),Adipose Subcutaneous,9.9610
3,3,ENSG00000196289.7,ENSG00000196289,BECN2,Adipose Subcutaneous,0.2881
4,4,ENSG00000197769.5,ENSG00000197769,MAP1LC3C,Adipose Subcutaneous,26.4800
...,...,...,...,...,...,...
651787,651787,ENSG00000078142.12,ENSG00000078142,PIK3C3,Adipose Subcutaneous,9.6290
651788,651788,ENSG00000090674.15,ENSG00000090674,MCOLN1(TRPML1),Adipose Subcutaneous,47.6400
651789,651789,ENSG00000130734.9,ENSG00000130734,ATG4D,Adipose Subcutaneous,12.0700
651790,651790,ENSG00000101460.12,ENSG00000101460,MAP1LC3A,Adipose Subcutaneous,37.8400


In [70]:
def renameTissue(x):
    if (x=='Adipose  Subcutaneous')|(x=='Adipose  Visceral (Omentum)'):
        return 'Adipose tissue'
    elif (x=='Adrenal gland'):
        return 'Adrenal gland'
    elif (x=='Brain - Amygdala'):
        return 'Amygdala'
    elif (x=='Breast  Mammary Tissue'):
        return 'Breast'
    elif (x=='Brain - Caudate (basal ganglia)'):
        return 'Caudate'
    elif (x=='Brain - Cerebellar Hemisphere') |(x=='Brain  Cerebellum'):
        return 'Cerebellum'
    elif (x=='Brain - Anterior cingulate cortex (BA24)')|(x=='Brain  Cortex')|(x=='Brain - Frontal Cortex (BA9)'):
        return 'Cerebral cortex'
    elif (x=='Cervix  Ectocervix') |(x=='Cervix  Endocervix'):
        return 'Cervix, uterine'
    elif (x=='Colon  Sigmoid') |(x=='Colon  Transverse'):
        return 'Colon'
    elif (x=='Uterus - Endometrium'):
        return 'Endometrium'
    elif (x=='Esophagus  Gastroesophageal Junction')|(x=='Esophagus  Mucosa')|(x=='Esophagus  Muscularis'):
        return 'Esophagus'
    elif (x=='Fallopian tube'):
        return 'Fallopian tube'
    elif (x=='Heart  Atrial Appendage ') |(x=='Heart  Left Ventricle'):
        return 'Heart muscle'
    elif (x=='Brain - Hippocampus'):
        return 'Hippocampus'
    elif (x=='Brain - Hypothalamus'):
        return 'Hypothalamus'
    elif (x=='Kidney  Cortex'):
        return 'Kidney'
    elif (x=='Brain - Nucleus accumbens (basal ganglia)'):
        return 'Nucleus accumbens'
    elif (x=='Pituitary gland'):
        return 'Pituitary gland'
    elif (x=='Brain - Putamen (basal ganglia)'):
        return 'Putamen'
    elif (x=='Salivary gland'):
        return 'Salivary gland'
    elif (x=='Muscle  Skeletal'):
        return 'Skeletal muscle'
    elif (x=='Skin  Not Sun Exposed (Suprapubic)')|(x=='Skin  Sun Exposed (Lower leg)'):
        return 'Skin'
    elif (x=='Small Intestine - Terminal Ileum'):
        return 'Small intestine'
    elif (x=='Brain - Spinal cord (cervical c-1)'):
        return 'Spinal cord'
    elif (x=='Brain - Substantia nigra'):
        return 'Substantia nigra'
    elif (x=='Thyroid gland'):
        return 'Thyroid gland'
    elif (x=='Bladder'):
        return 'Urinary bladder'
    else:
        return x

In [71]:
autophagy_geneTPM['Tissue']=autophagy_geneTPM['Tissue'].apply(lambda x: renameTissue(x))

In [72]:
autophagy_geneTPM.head()

,Unnamed: 0,gene_id,new_gene_id,GeneName,Tissue,GETx_TPM
0,0,ENSG00000125703.14,ENSG00000125703,ATG4C,Adipose tissue,5.3030
1,1,ENSG00000153898.12,ENSG00000153898,MCOLN2(TRPML2),Adipose tissue,3.5700
2,2,ENSG00000055732.12,ENSG00000055732,MCOLN1(TRPML3),Adipose tissue,9.9610
3,3,ENSG00000196289.7,ENSG00000196289,BECN2,Adipose tissue,0.2881
4,4,ENSG00000197769.5,ENSG00000197769,MAP1LC3C,Adipose tissue,26.4800


In [73]:
url_sample = autophagy_geneTPM[['new_gene_id','Tissue','GeneName']]

In [75]:
gene_id = url_sample.drop_duplicates('new_gene_id',keep='first')['new_gene_id'].values.tolist()

In [76]:
tissue = ['duodenum',
'Endometrium',
'Epididymis',
'Gallbladder',
'Lymph node',
'Parathyroid gland',
'Placenta',
'Prostate',
'Rectum',
'Seminal vesicle',
'Small intestine',
'Smooth muscle',
'Tonsil']


In [77]:
url_sample['url'] = url_sample['new_gene_id']+'-'+url_sample['GeneName']
url_sample['Tissue']=url_sample['Tissue'].apply(lambda x: x.lower())
url_sample['Tissue-url'] =url_sample['Tissue'].apply(lambda x: x.split())
url_sample['Tissue-url'] =url_sample['Tissue-url'].apply(lambda x: '+'.join(x))

<ipython-input-77-b70b7b83fd73>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  url_sample['url'] = url_sample['new_gene_id']+'-'+url_sample['GeneName']
<ipython-input-77-b70b7b83fd73>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  url_sample['Tissue']=url_sample['Tissue'].apply(lambda x: x.lower())
<ipython-input-77-b70b7b83fd73>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [78]:
url_sample['Tissue-url']=url_sample['Tissue-url'].apply(lambda x: x.replace(',','%2C'))
 

<ipython-input-78-5d943c59c267>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  url_sample['Tissue-url']=url_sample['Tissue-url'].apply(lambda x: x.replace(',','%2C'))


In [80]:
url_sample['url'] = 'https://v18.proteinatlas.org/' + url_sample['url'] + '/tissue/' +url_sample['Tissue-url'] +'#rnaseq'

<ipython-input-80-efd13ce3c572>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  url_sample['url'] = 'https://v18.proteinatlas.org/' + url_sample['url'] + '/tissue/' +url_sample['Tissue-url'] +'#rnaseq'


In [81]:
autophagyV8Url=url_sample['url'].values.tolist()

In [85]:
import re
autophagyV8Url_new=[]

for i in autophagyV8Url:
    
    i = re.sub(r'\([^()]*\)', '', i)
    autophagyV8Url_new.append(i)

In [87]:
np.save('/Users/xinyutang/Documents/HPA2/autophagyV8Url.npy',autophagyV8Url_new)

# HPA Tissue Data

In [119]:
autophagyV8Url_new = np.load('/Users/xinyutang/Documents/HPA2/autophagyV8Url.npy')

In [98]:
i.split('/')[3].split('-')[1]

'ATG4C'

In [120]:
new_gene = ['SQSTM1',
'CALCOCO2',
'NBR1',
'OPTN',
'TAX1BP1',
'NCOA4',
'RB1CC1',
'BECN1',
'MCOLN2',    
'MCOLN3']
l=[]
for i in autophagyV8Url_new:
    a = i.split('/')[3].split('-')[1]
    if a in new_gene:
        l.append(i)

In [121]:
l=set(l)

In [130]:

np.save('/Users/xinyutang/Documents/HPA2/autophagyV8Url_added.npy',list(l))


In [107]:
dic2 = dic.item()
gene = list(dic2.keys())[0]
tissue = list(dic2[gene].keys())[0]
TPM = dic2[gene][tissue]
TPM = [x.split(': ')[1] for x in TPM]
df2 = pd.DataFrame({'new_gene_id': gene, 'tissue': tissue, 'TPM':TPM})

In [132]:
path = '/Users/xinyutang/Documents/HPA2/Data/HPA-V8/'
l=[]
for r, d, f in tqdm(os.walk(path)):
    for file in f:
        file_path = path+file
        print(file_path)
        dic = np.load(file_path,allow_pickle=True)
        dic2 = dic.item()
        gene = list(dic2.keys())[0]
        tissue = list(dic2[gene].keys())[0]
        TPM = dic2[gene][tissue]
        TPM = [x.split(': ')[1] for x in TPM]
        df2 = pd.DataFrame({'new_gene_id': gene, 'tissue': tissue, 'TPM':TPM})
        l.append(df2)


0it [00:00, ?it/s]

/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000034713-GABARAPL2small intestine.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000136436-CALCOCO2placenta.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000140941-MAP1LC3Btestis.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000123240-OPTNlymph node.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000123240-OPTNkidney.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000070540-WIPI1kidney.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000101844-ATG4Afallopian tube.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000078142-PIK3C3heart atrial appendage.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000198925-ATG9Anerve tibial.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000170296-GABARAPpancreas.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000123395-ATG101prostate.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000070540-WIPI1epididymis.npy
/Users/xinyutang/Docume

/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000152348-ATG10gallbladder.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000023287-RB1CC1lymph node.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000066739-ATG2Bartery coronary.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000066739-ATG2Bseminal vesicle.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000152348-ATG10urinary bladder.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000197548-ATG7gallbladder.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000140474-ULK3salivary gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000188554-NBR1artery aorta.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000198925-ATG9Abreast.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000153898-MCOLN2skin.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000144848-ATG3lung.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000168397-ATG4Bthyroid gland.npy
/Users/xinyutang/Documents/HPA2/

/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000266412-NCOA4cerebellum.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000197548-ATG7lymph node.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000198925-ATG9Askeletal muscle.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000123395-ATG101endometrium.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000112561-TFEBtonsil.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000126775-ATG14breast.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000034713-GABARAPL2endometrium.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000140474-ULK3kidney.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000197548-ATG7placenta.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000130734-ATG4Dcerebral cortex.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000157954-WIPI2artery tibial.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000157954-WIPI2lung.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/

/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000125703-ATG4Cparathyroid gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000023287-RB1CC1cervix%2C uterine.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000153898-MCOLN2nerve tibial.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000123240-OPTNadrenal gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000101844-ATG4Athyroid gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000066739-ATG2Bpancreas.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000266412-NCOA4heart atrial appendage.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000106052-TAX1BP1nerve tibial.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000130734-ATG4Dspleen.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000140474-ULK3artery aorta.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000106052-TAX1BP1adrenal gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000157954-WIPI2duodenum.npy
/Us

...

/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000123395-ATG101adrenal gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000145016-RUBCNuterus.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000110046-ATG2Alung.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000168397-ATG4Bsalivary gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000145016-RUBCNcolon.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000170296-GABARAPheart atrial appendage.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000168397-ATG4Bovary.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000126581-BECN1heart muscle.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000051382-PIK3CBsmall intestine.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000175224-ATG13testis.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000145016-RUBCNskin.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000051382-PIK3CBrectum.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8

/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000139112-GABARAPL1artery aorta.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000070540-WIPI1prostate.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000123240-OPTNcervix%2C uterine.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000157954-WIPI2cerebral cortex.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000161011-SQSTM1heart muscle.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000057663-ATG5adipose tissue.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000145782-ATG12heart muscle.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000101844-ATG4Askeletal muscle.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000152348-ATG10vagina.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000023287-RB1CC1pituitary gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000177169-ULK1skin.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000110046-ATG2Aartery aorta.npy
/Users/xinyutan

/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000197548-ATG7liver.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000057663-ATG5cervix%2C uterine.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000078142-PIK3C3stomach.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000085978-ATG16L1breast.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000188554-NBR1adipose tissue.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000144848-ATG3urinary bladder.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000023287-RB1CC1colon.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000145016-RUBCNtestis.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000078142-PIK3C3lung.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000197548-ATG7adipose tissue.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000153898-MCOLN2ovary.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000197548-ATG7endometrium.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG000

1it [00:02,  2.55s/it]

/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000034713-GABARAPL2heart atrial appendage.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000163625-WDFY3artery tibial.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000197769-MAP1LC3Cskin.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000023287-RB1CC1liver.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000110046-ATG2Asalivary gland.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000175224-ATG13pancreas.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000101844-ATG4Atestis.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000266412-NCOA4urinary bladder.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000170296-GABARAPvagina.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000023287-RB1CC1cerebral cortex.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000023287-RB1CC1duodenum.npy
/Users/xinyutang/Documents/HPA2/Data/HPA-V8/ENSG00000051382-PIK3CBartery tibial.npy
/Users/xinyutang/D

In [133]:
hpa_TPM = pd.concat(l,axis=0).reset_index(drop=True)

In [134]:
hpa_TPM[['new_gene_id','GeneName']] = hpa_TPM.new_gene_id.str.split("-",expand=True,)

In [136]:
hpa_TPM=hpa_TPM.rename({'tissue':'Tissue'},axis=1)

In [143]:
hpa_TPM.GeneName=hpa_TPM.GeneName.apply(lambda x: 'SQSTM1(p62)' if x=='SQSTM1' else\
                                            'CALCOCO2 (NDP52)' if x =='CALCOCO2'else\
                                        'B1CC1(FIP200)' if x=='B1CC1'else\
                                         'MCOLN1(TRPML1)' if x=='MCOLN1'else\
                                        'MCOLN2(TRPML2)' if x=='MCOLN2'else\
                                        'MCOLN3(TRPML3) ' if x=='MCOLN3' else x)


In [148]:
rna_tissue.GeneName=rna_tissue.GeneName.apply(lambda x: 'MCOLN3(TRPML3) ' if x=='MCOLN1(TRPML3)' else x)



In [146]:
hpa_TPM.to_csv('/Users/xinyutang/Desktop/R/Project/P9/autophagy_hpa_TPM.csv')

In [149]:
rna_tissue.to_csv('/Users/xinyutang/Documents/HPA2/Data/GETxV8/gtex_autophogy_TPM_final.csv')

# Merge Dataset

In [150]:
#Upload Gtex data
rna_tissue=pd.read_csv('/Users/xinyutang/Documents/HPA2/Data/GETxV8/gtex_autophogy_TPM_final.csv')
#Uplod hpa data
rna_tissue_hpa = pd.read_csv('/Users/xinyutang/Desktop/R/Project/P9/autophagy_hpa_TPM.csv')


In [153]:
del rna_tissue['Unnamed: 0']
del rna_tissue['Unnamed: 0.1']

In [154]:
rna_tissue.head()

,gene_id,new_gene_id,GeneName,Tissue,GETx_TPM
0,ENSG00000125703.14,ENSG00000125703,ATG4C,Adipose Subcutaneous,5.3030
1,ENSG00000153898.12,ENSG00000153898,MCOLN2(TRPML2),Adipose Subcutaneous,3.5700
2,ENSG00000055732.12,ENSG00000055732,MCOLN3(TRPML3),Adipose Subcutaneous,9.9610
3,ENSG00000196289.7,ENSG00000196289,BECN2,Adipose Subcutaneous,0.2881
4,ENSG00000197769.5,ENSG00000197769,MAP1LC3C,Adipose Subcutaneous,26.4800


In [151]:
rna_tissue_hpa.head()

,Unnamed: 0,new_gene_id,Tissue,TPM,GeneName
0,0,ENSG00000034713,small intestine,64.4,GABARAPL2
1,1,ENSG00000034713,small intestine,83.8,GABARAPL2
2,2,ENSG00000034713,small intestine,78.1,GABARAPL2
3,3,ENSG00000034713,small intestine,77.0,GABARAPL2
4,4,ENSG00000136436,placenta,65.9,CALCOCO2 (NDP52)


In [155]:
rna_tissue['Dataset'] ='GETx'

In [156]:
rna_tissue =rna_tissue.rename({'GETx_TPM':'TPM'},axis=1)

In [158]:
del rna_tissue_hpa['Unnamed: 0']

In [157]:
rna_tissue_hpa['Dataset'] ='HPA'

In [159]:
rna_tissue_hpa =rna_tissue_hpa.rename({'hpa_TPM':'TPM'},axis=1)

In [160]:
rna_tissue=rna_tissue[['new_gene_id','Tissue','TPM','GeneName','Dataset']]

In [161]:
tissue_merge = pd.concat([rna_tissue,rna_tissue_hpa],axis=0)

In [162]:
tissue_merge['Tissue']=tissue_merge['Tissue'].apply(lambda x: x.lower())

In [163]:
tissue_merge

,new_gene_id,Tissue,TPM,GeneName,Dataset
0,ENSG00000125703,adipose subcutaneous,5.3030,ATG4C,GETx
1,ENSG00000153898,adipose subcutaneous,3.5700,MCOLN2(TRPML2),GETx
2,ENSG00000055732,adipose subcutaneous,9.9610,MCOLN3(TRPML3),GETx
3,ENSG00000196289,adipose subcutaneous,0.2881,BECN2,GETx
4,ENSG00000197769,adipose subcutaneous,26.4800,MAP1LC3C,GETx
...,...,...,...,...,...
6954,ENSG00000106052,liver,46.0000,TAX1BP1,HPA
6955,ENSG00000106052,liver,39.4000,TAX1BP1,HPA
6956,ENSG00000106052,liver,30.4000,TAX1BP1,HPA
6957,ENSG00000106052,liver,33.0000,TAX1BP1,HPA


In [164]:
tissue_class ={'adipose  subcutaneous':'adipose tissue',
                   'adipose  visceral (omentum)':'adipose tissue',
                   'bladder':'Urinary bladder',
                   'brain  cerebellum':'cerebellum',
                   'brain  cortex':'cerebral cortex',
                   'breast  mammary tissue':'breast',
                   'cervix  ectocervix':'cervix,uterine',
                   'cervix  endocervix':'cervix,uterine',
                   'cervix%2c uterine':'cervix,uterine',
                   'colon  sigmoid':'colon',
                   'colon  transverse':'colon',
                   'esophagus  gastroesophageal junction':'esophagus',
                   'esophagus  mucosa':'esophagus',
                   'esophagus  muscularis':'esophagus',
                   'heart  atrial appendage':'heart muscle',
                   'heart  left ventricle':'heart muscle',
                   'kidney  cortex':'kidney',
                   'kidney  medulla':'kidney',
                   'minor salivary gland':'salivary gland',
                   'muscle  skeletal':'skeletal muscle',
                   'skin  not sun exposed (suprapubic)':'skin',
                    'skin  sun exposed (lower leg)':'skin',
                   'thyroid':'thyroid gland',
                   'uterus':'endometrium',
                  'duodenum':'epithelium',
               'parathyroid gland':'epithelium',
                'rectum':'epithelium',
                   'gallbladder':'epithelium',
              'small intestine':'epithelium',
              'epididymis':'epithelium',
              'tonsil':'lymph node'}


In [165]:
tissue_merge = tissue_merge.replace({"Tissue": tissue_class})

In [167]:
tissue_merge.to_csv('/Users/xinyutang/Desktop/R/Project/P9/autophagy_hpa_gtex.csv')

In [168]:
tissue_merge['GeneName']=tissue_merge['GeneName'].apply(lambda x: x.upper())

In [169]:
gtex = tissue_merge[tissue_merge['Dataset']=='GETx']
hpa = tissue_merge[tissue_merge['Dataset']=='HPA']

In [170]:
gtex_unique=gtex.drop_duplicates(['GeneName','Tissue'],keep='first')
gtex_unique = gtex_unique[['GeneName','Tissue','Dataset']]

In [171]:
hpa_unique=hpa.drop_duplicates(['GeneName','Tissue'],keep='first')
hpa_unique = hpa_unique[['GeneName','Tissue','Dataset']]

In [172]:
outerJoin = pd.merge(gtex_unique, hpa_unique, on=['GeneName','Tissue'], how='outer')

In [173]:
gtex=gtex[['GeneName','Tissue','TPM']]
df1 = pd.merge(outerJoin,gtex,on=['GeneName','Tissue'],how='left').rename({'TPM':'gtex_TPM'},axis=1)
hpa=hpa[['GeneName','Tissue','TPM']]
df2 = pd.merge(outerJoin,hpa,on=['GeneName','Tissue'],how='left').rename({'TPM':'hpa_TPM'},axis=1)

In [190]:
df1.to_csv('/Users/xinyutang/Desktop/R/Project/P11/autophagy_gtex_outerjoin.csv')

In [191]:
df2.to_csv('/Users/xinyutang/Desktop/R/Project/P11/autophagy_hpa_outerjoin.csv')

In [187]:
df1['Tissue']=df1['Tissue'].apply(lambda x: x.upper())

In [189]:
df2['Tissue']=df2['Tissue'].apply(lambda x: x.upper())